# Класс для работы с базой, находящихся в конкретной области людей

In [80]:
import pickle as pkl
import json as js
import os
import numpy as np
from scipy.spatial import distance

In [152]:
class Dataset:
    def __init__(self, buffer_file):
        self.buf = buffer_file
        if not os.path.exists(self.buf):
            with open(self.buf, 'w') as f:
                f.write(js.dumps([{},]))
    def __call__(self, person_info, action):
        if action:
            self._input(person_info)
        else:
            self._output(person_info)
    
    def dump(self, data):
        with open(self.buf, 'w') as f:
            f.write(js.dumps(data))
    def load(self):
        return js.load(open(self.buf))
    def _input(self, person_info):
        inarea = self.load()
        if not self.__inarea__(inarea, person_info)[0]:
            inarea.append(person_info)
        self.dump(inarea)
    def _output(self, person_info):
        inarea = self.load()
        check = self.__inarea__(inarea, person_info)
        if check[0]:
            inarea.remove(inarea[check[1]])
        self.dump(inarea)
    def __inarea__(self, inarea, target_person):
        vectors = [self.__strToArray__(p['vector']) for p in inarea]
        dists = []
        for v in vectors:
            dists.append(distance.euclidean(v, self.__strToArray__(target_person['vector'])))
        dists = np.asarray(dists)
        if np.min(dists) < 0.55:
            return (True, np.argmin(dists))
        else:
            return (False, 0)
        
    def __strToArray__(self, string):
        return np.asarray([float(v) for v in string.split(';')])
    
    def fictionvectors(self):
        fict_vs = []
        for i in range(10):
            fict_vs.append({'vector':self.__arrayToStr__(np.random.random(24))})
        with open(self.buf, 'w') as f:
            f.write(js.dumps(fict_vs))
            
    def __arrayToStr__(self, array):
        array_strs = [str(v) for v in array]
        return ';'.join(array_strs)

In [153]:
db = Dataset("inarea.json")
#db._input({'none':None})
db.fictionvectors()
#db._input({'vector':db.__arrayToStr__(np.random.random(24))})
db({'vector':db.__arrayToStr__(np.random.random(24))}, False)